# Perplexity playbook

<p>
Mal Minhas, v0.3<br>
15.01.24
</p>

## 1. Introduction

Perplexity is an intelligent search engine with a chatbot sytle interface that leverages popular AI models to offer comprehensive answers to user inquiries.  It is able to search the internet to provide real-time information via a friendly user experience that includes native mobile apps.  The tool is specifically designed to perform as a research companion providing links to underlying source articles unlike ChatGPT in default mode.  Perplexity comes with an API called [`pplx-api`](https://blog.perplexity.ai/blog/introducing-pplx-api) which was introduced in October 2023.  See [here](https://docs.perplexity.ai/docs/getting-started) for a guide on how to set up an API key and get started with `pplx-api`.  The following examples assume you have set up the API key in an environment variable called `PERPLEXITY_API_KEY`.

Supported models are [as follows](https://docs.perplexity.ai/docs/model-cards):
```
Model	                  Context Length       Model Type
codellama-34b-instruct	      16384          Chat Completion
llama-2-70b-chat               4096          Chat Completion
mistral-7b-instruct [2]	       4096 [1]      Chat Completion
mixtral-8x7b-instruct	       4096 [1]      Chat Completion
pplx-7b-chat	               8192          Chat Completion
pplx-70b-chat	               4096          Chat Completion
pplx-7b-online	               4096          Chat Completion
pplx-70b-online	               4096          Chat Completion

[1] We will be increasing the context length of mistral-7b-instruct to 32k tokens (see roadmap).
[2] This model refers to the v0.2 release of mistral-7b-instruct.
```

The only API function currently supported is on `chat_completions` documented [here](https://docs.perplexity.ai/reference/post_chat_completions).  The next section outlines how to use it.

## 2. Basic Usage

Let's start with a few basics including a utility function to render text for display in Jupyter:

In [1]:
import os
import requests
from IPython.display import Markdown, display

def renderCompletion(text: str):
    display(Markdown(text))
    
API_KEY = os.environ.get("PERPLEXITY_API_KEY")
DEFAULT_MODEL = "mistral-7b-instruct"
DEFAULT_TOKENS = 1024
CONCISE_DETAIL = "Be precise and concise."
MID_LEVEL_DETAIL = "Explain in a few sentences."
GREAT_DETAIL = "Explain in a lot of detail."
POLITE_DETAIL = "You are an artificial intelligence assistant and you need to \engage in a helpful, detailed, polite conversation with a user."
DEFAULT_TEMP = 0.0

The following is a convenience function that wraps the Perplexity API allowing different models to be used and also differing levels of depth of response detail:

In [2]:
def getPerplexityResponse(question:str, model:str=DEFAULT_MODEL, direction:str=MID_LEVEL_DETAIL, max_tokens:int=DEFAULT_TOKENS, temperature:float=DEFAULT_TEMP) -> str:
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": False,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": f"{direction}"
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    response = requests.post(url, json=payload, headers=headers).json()
    return renderCompletion(response.get("choices")[0].get("message").get("content"))

Let's test the function on an example query using the default Mistral 7b model and then Llama2.  Note the differences in response timing:

In [3]:
%%time

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
getPerplexityResponse(prompt)

In Southampton, UK, if you have a broken washing machine, you should call a professional appliance repair technician. They have the necessary skills and expertise to diagnose and fix common washing machine issues. You can look up local appliance repair services online or check directories like Yell or Thomson Local. Alternatively, you could contact a local appliance repair company, such as Southampton Appliance Repairs or A1 Appliance Repairs, for assistance. Make sure to describe the problem accurately when you call to help them prepare for the repair visit

CPU times: user 17 ms, sys: 5 ms, total: 22 ms
Wall time: 1.67 s


In [4]:
%%time

getPerplexityResponse(prompt, model='llama-2-70b-chat')

If you have a broken washing machine in Southampton, UK, you can call a local appliance repair service or a professional appliance engineer to fix the issue. You can search online for appliance repair services in your area, or look up local trade directories such as Yell or Checkatrade to find a reputable repair person. Additionally, you can also contact the manufacturer's customer service number to see if they offer repair services or can recommend a local technician

CPU times: user 20 ms, sys: 4.89 ms, total: 24.9 ms
Wall time: 2.24 s


In [5]:
%%time

getPerplexityResponse(prompt, model='codellama-34b-instruct', direction=POLITE_DETAIL)

I'm so sorry to hear that your washing machine is broken! There are a few options you can consider to get it mended.

1. Contact the manufacturer: If your washing machine is still under warranty, you may be able to contact the manufacturer for repair or replacement options. They may be able to provide you with a repair kit or send a technician to your location to fix the machine.
2. Call a repair service: There are many repair services that specialize in fixing washing machines. You can search online for "washing machine repair" in your area and compare prices and reviews to find a reputable service.
3. Contact a plumber: If your washing machine is not under warranty, you may need to call a plumber to fix the issue. Plumbers often have experience with fixing washing machines and other appliances.
4. Consider purchasing a new machine: If your washing machine is old or has been in continuous use for a long time, it may be more cost-effective to purchase a new machine rather than trying to repair it.

In Southampton, you can call the following trade professionals for assistance:

* Washing machine repair services:
	+ Washing Machine Repair Southampton: 023 8066 1111
	+ Southampton Washing Machine Repair: 023 8066 2222
* Plumbers:
	+ Southampton Plumbing: 023 8066 3333
	+ Plumbing Southampton: 023 8066 4444

I hope this information is helpful! If you have any further questions, please don't hesitate to ask

CPU times: user 25.8 ms, sys: 5.78 ms, total: 31.6 ms
Wall time: 5.52 s


Let's see how it handles a query with a more detailed response:

In [6]:
%%time

prompt = """"I have a broken garden gate which is no longer closing properly.  
What trade do I need to call to fix it?"""
getPerplexityResponse(prompt, direction=GREAT_DETAIL)

A broken garden gate that is no longer closing properly is an issue that requires the expertise of a tradesperson specializing in carpentry, specifically in gate repair or installation. Here's a detailed explanation of why this is the case and what you can expect when you call a carpenter to fix the problem.

Carpentry is a skilled trade that involves the design, construction, and repair of structures made of wood, as well as other materials like metal and plastic. Carpenters are trained to work with various tools and techniques to create, install, and maintain structures, including doors and gates.

When it comes to a garden gate, there are several components that can malfunction, leading to it not closing properly. These components include the hinges, the latch or lock, the posts or frames, and the gate itself.

Hinges are the metal fittings that attach the gate to the posts or walls. Over time, hinges can become loose, rusted, or worn out, causing the gate to sag or not close properly. A carpenter can assess the condition of the hinges and either tighten or replace them as needed.

The latch or lock is the mechanism that secures the gate when it's closed. If the gate is not closing properly due to a faulty latch or lock, a carpenter can inspect the issue and make any necessary repairs or replacements.

The posts or frames that support the gate can also be a source of the problem. If the posts are rotten, uneven, or have shifted, the gate may not close properly. A carpenter can assess the condition of the posts and either repair or replace them as needed.

Lastly, the gate itself may be the issue. If the gate is warped, bent, or otherwise damaged, it may not close properly. A carpenter can assess the condition of the gate and either repair or replace it as needed.

When you call a carpenter to fix your broken garden gate, they will typically follow these steps:

1. Inspection: The carpenter will inspect the gate and identify the cause of the problem.
2. Quote: Based on the inspection, the carpenter will provide you with a quote for the repairs or replacement.
3. Repairs or Replacement: The carpenter will then carry out the repairs or replacement as agreed.
4. Testing: Once the repairs or replacement are complete, the carpenter will test the gate to ensure it's closing properly.
5. Cleanup: The carpenter will clean up the work area and leave your garden looking neat and tidy.

In summary, if you have a broken garden gate that is no longer closing properly, you should call a carpenter to fix it. Carpenters are trained to assess and repair various components of gates, including hinges, latches, locks, posts, and the gate itself. They will follow a process of inspection, quote, repairs or replacement, testing, and cleanup to ensure your garden gate is functioning properly once again

CPU times: user 24.8 ms, sys: 7.43 ms, total: 32.2 ms
Wall time: 6.23 s


## 3. Streaming interface

tbd

In [7]:
%%time

def getPerplexityStreamingResponse(question, model=DEFAULT_MODEL, direction=MID_LEVEL_DETAIL, max_tokens=DEFAULT_TOKENS, temperature=DEFAULT_TEMP):
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": True,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": direction,
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    print('start')
    response_stream = requests.post(url, json=payload, headers=headers)
    print('end')
    for response in response_stream:
        print(len(response))
    #return response.get("choices")[0].get("message").get("content")

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
#getPerplexityStreamingResponse(prompt, direction=GREAT_DETAIL)

CPU times: user 9 µs, sys: 1 µs, total: 10 µs
Wall time: 13.8 µs
